강의자료
https://github.com/smu405/e  
이더리움
http://remix.ethereum.org/

## cd Users\minji\Code\201810808

# cmd 에서 명령어 모음(in 학번 디렉토리

### __geth 접속하기__  
_gethNow.bat  
이후 비밀번호 입력
<br>

### geth 콘솔 열기
geth attach http://localhost:8445


# geth 콘솔 명령어(geth 콘솔 실행 후 cmd)
## 주피터에서 실행할 때는 
`!geth -exec eth.accounts attach http://localhost:8445`  
이런식으로


In [1]:
!geth -exec eth.accounts attach http://localhost:8445

["0x8e71085ffefccb88a4b54d4c93a866db7928a486", "0x9eabca74ecd2d27638af3a8ee7532b7853a0310d", "0x223bd75a28f01aad68f309fce60a22b06252d420"]


### 계정 확인
eth.accounts
<br>

### 각각 잔고(잔액) 확인하기
eth.getBalance(eth.accounts[0]);
<br>

### 노드 정보 확인하기
admin.nodeinfo

### 충전하기
miner.start()  
miner.stop()

### 트랜잭션 확인
eth.pendingTransactions  

### authentication 오류 뜰 때
personal.unlockAccount(eth.coinbase); 하고 비밀번호 입력

### mining 하기(pending 해소)
miner.start(1);admin.sleepBlocks(1);miner.stop();

### node로 js실행(명령창
C:\Users\minji\Code\201810808>node src/OrderEventUse.js

### 가나슈 실행
`node_modules\.bin\ganache-cli.cmd -p 8345`


# 주피터에서 하는 명령어(node나 solc)

# 작년 기말고사
### 문제 2: 주문을 구현하세요.
1번 계정을 manager로 설정하여 다음 주문내역을 구현:
```
- 주문번호
- 주문상품
- 주문개수
- 주문일자
- 주문금액
- 상태: Ordered, Delivered, Received, Closed, Returned 등
```

1) **모두 다른 계정을 사용하여** 주문갯수 3건을 입력, 주문하면서 금액 전송한다.
```
305, water, 12, 12000
306, burger, 10, 10000
307, paper, 9, 28000
```

2) 주문의 총액 출력

3) 2번 계정의 주문내역 (주문번호, 주문상품, 금액, 상태) 출력

4) 주문번호로 주문내역 (주문번호, 주문상품, 금액, 상태) 출력

5) 컨트랙의 잔고 조회

6) 2번째 주문을 배송한 후 'Delivered'로 출력 (manager만 가능)

In [37]:
%%writefile src/Manager.sol
pragma solidity 0.5.1;

contract Manager{
    struct OrdersMap{
        uint orderNumber;//주문번호
        string orderProduct;//주문상품
        uint numProduct;//주문개수
        uint256 orderTime;//주문일자
        uint orderBalance; //주문금액
    }
    address payable owner;
    //bidrectional map
    mapping(address=>OrdersMap) memberMap;//key:address, 키를 넣으면 OrdersMap struct이 반환
    mapping(uint=>address) addressByOM;//address by orderNumber, ordernumber이용해서 주소를 구한다.
    mapping (address => string) status;
    mapping (address => uint) balanceOf;
    
    constructor() public {
        owner = msg.sender;
    }
    
    function Order(address user, uint ordernum, string memory product, uint count, uint _cost) public payable {
       OrdersMap memory m=OrdersMap(ordernum, product, count, now, _cost);
       memberMap[user]=m;
       addressByOM[ordernum]=user;
       status[user]="Ordered";
       balanceOf[owner]+=_cost;
    }
    
    
    function updateStatus(address user)public onlyManager returns(string memory) {
        status[user]="Delivered";
        return status[user];
    }
    
    function getsOrderWithAddr(address user) view public returns (uint, string memory, uint, string memory){
        OrdersMap memory u=memberMap[user];
        return(u.orderNumber, u.orderProduct, u.orderBalance, status[user]);
    }
    
    function getOrderWithNum(uint odnum) view public returns (uint, string memory, uint, string memory){
        OrdersMap memory u=memberMap[addressByOM[odnum]];
        return(u.orderNumber, u.orderProduct, u.orderBalance, status[addressByOM[odnum]]);
    }
    
    function getBalance() public view returns(uint) {//잔고 확인, state 읽으니까 view
        return (address(this).balance);
    }
    
    function ownerBalance() public view returns(uint) {//잔고 확인, state 읽으니까 view
        return (balanceOf[owner]);
    }
    
    modifier onlyManager {
        require(msg.sender == owner);
        _;
    }
    
}

Overwriting src/Manager.sol


In [38]:
!solc --gas --abi --bin src/Manager.sol


======= src/Manager.sol:Manager =======
Gas estimation:
construction:
   21097 + 795200 = 816297
external:
   Order(address,uint256,string,uint256,uint256):	infinite
   getBalance():	623
   getOrderWithNum(uint256):	infinite
   getsOrderWithAddr(address):	infinite
   ownerBalance():	810
   updateStatus(address):	infinite
Binary: 
608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff160217905550610f88806100606000396000f3fe608060405260043610610072576000357c01000000000000000000000000000000000000000000000000000000009004806301162b931461007757806312065fe014610141578063258b813c1461016c57806383b6bc2614610265578063bedcf003146103a9578063d149ebda146103d4575b600080fd5b34801561008357600080fd5b506100c66004803603602081101561009a57600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff169060200190929190505050610502565b6040518080602001828103825283818151815260200191508051906020019080838360005b838

In [71]:
%%writefile src/ManagerDeploy.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8445"));
var _abiArray =[{"constant":false,"inputs":[{"name":"user","type":"address"}],"name":"updateStatus","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"user","type":"address"},{"name":"ordernum","type":"uint256"},{"name":"product","type":"string"},{"name":"count","type":"uint256"},{"name":"_cost","type":"uint256"}],"name":"Order","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":true,"inputs":[{"name":"user","type":"address"}],"name":"getsOrderWithAddr","outputs":[{"name":"","type":"uint256"},{"name":"","type":"string"},{"name":"","type":"uint256"},{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"ownerBalance","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[{"name":"odnum","type":"uint256"}],"name":"getOrderWithNum","outputs":[{"name":"","type":"uint256"},{"name":"","type":"string"},{"name":"","type":"uint256"},{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"}];
var _bin="0x" + "608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff160217905550610f88806100606000396000f3fe608060405260043610610072576000357c01000000000000000000000000000000000000000000000000000000009004806301162b931461007757806312065fe014610141578063258b813c1461016c57806383b6bc2614610265578063bedcf003146103a9578063d149ebda146103d4575b600080fd5b34801561008357600080fd5b506100c66004803603602081101561009a57600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff169060200190929190505050610502565b6040518080602001828103825283818151815260200191508051906020019080838360005b838110156101065780820151818401526020810190506100eb565b50505050905090810190601f1680156101335780820380516001836020036101000a031916815260200191505b509250505060405180910390f35b34801561014d57600080fd5b506101566106c7565b6040518082815260200191505060405180910390f35b610263600480360360a081101561018257600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff16906020019092919080359060200190929190803590602001906401000000008111156101c957600080fd5b8201836020820111156101db57600080fd5b803590602001918460018302840111640100000000831117156101fd57600080fd5b91908080601f016020809104026020016040519081016040528093929190818152602001838380828437600081840152601f19601f82011690508083019250505050505050919291929080359060200190929190803590602001909291905050506106e6565b005b34801561027157600080fd5b506102b46004803603602081101561028857600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff1690602001909291905050506108f1565b604051808581526020018060200184815260200180602001838103835286818151815260200191508051906020019080838360005b838110156103045780820151818401526020810190506102e9565b50505050905090810190601f1680156103315780820380516001836020036101000a031916815260200191505b50838103825284818151815260200191508051906020019080838360005b8381101561036a57808201518184015260208101905061034f565b50505050905090810190601f1680156103975780820380516001836020036101000a031916815260200191505b50965050505050505060405180910390f35b3480156103b557600080fd5b506103be610b15565b6040518082815260200191505060405180910390f35b3480156103e057600080fd5b5061040d600480360360208110156103f757600080fd5b8101908080359060200190929190505050610b7d565b604051808581526020018060200184815260200180602001838103835286818151815260200191508051906020019080838360005b8381101561045d578082015181840152602081019050610442565b50505050905090810190601f16801561048a5780820380516001836020036101000a031916815260200191505b50838103825284818151815260200191508051906020019080838360005b838110156104c35780820151818401526020810190506104a8565b50505050905090810190601f1680156104f05780820380516001836020036101000a031916815260200191505b50965050505050505060405180910390f35b60606000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff1614151561055f57600080fd5b6040805190810160405280600981526020017f44656c6976657265640000000000000000000000000000000000000000000000815250600360008473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002090805190602001906105e7929190610e07565b50600360008373ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020019081526020016000208054600181600116156101000203166002900480601f0160208091040260200160405190810160405280929190818152602001828054600181600116156101000203166002900480156106bb5780601f10610690576101008083540402835291602001916106bb565b820191906000526020600020905b81548152906001019060200180831161069e57829003601f168201915b50505050509050919050565b60003073ffffffffffffffffffffffffffffffffffffffff1631905090565b6106ee610e87565b60a06040519081016040528086815260200185815260200184815260200142815260200183815250905080600160008873ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002060008201518160000155602082015181600101908051906020019061077e929190610eb7565b50604082015181600201556060820151816003015560808201518160040155905050856002600087815260200190815260200160002060006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff1602179055506040805190810160405280600781526020017f4f72646572656400000000000000000000000000000000000000000000000000815250600360008873ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020908051906020019061087a929190610e07565b5081600460008060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002060008282540192505081905550505050505050565b6000606060006060610901610e87565b600160008773ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002060a0604051908101604052908160008201548152602001600182018054600181600116156101000203166002900480601f0160208091040260200160405190810160405280929190818152602001828054600181600116156101000203166002900480156109ef5780601f106109c4576101008083540402835291602001916109ef565b820191906000526020600020905b8154815290600101906020018083116109d257829003601f168201915b5050505050815260200160028201548152602001600382015481526020016004820154815250509050806000015181602001518260800151600360008a73ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020829250808054600181600116156101000203166002900480601f016020809104026020016040519081016040528092919081815260200182805460018160011615610100020316600290048015610afe5780601f10610ad357610100808354040283529160200191610afe565b820191906000526020600020905b815481529060010190602001808311610ae157829003601f168201915b505050505090509450945094509450509193509193565b6000600460008060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002054905090565b6000606060006060610b8d610e87565b600160006002600089815260200190815260200160002060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002060a0604051908101604052908160008201548152602001600182018054600181600116156101000203166002900480601f016020809104026020016040519081016040528092919081815260200182805460018160011615610100020316600290048015610cae5780601f10610c8357610100808354040283529160200191610cae565b820191906000526020600020905b815481529060010190602001808311610c9157829003601f168201915b505050505081526020016002820154815260200160038201548152602001600482015481525050905080600001518160200151826080015160036000600260008c815260200190815260200160002060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020829250808054600181600116156101000203166002900480601f016020809104026020016040519081016040528092919081815260200182805460018160011615610100020316600290048015610df05780601f10610dc557610100808354040283529160200191610df0565b820191906000526020600020905b815481529060010190602001808311610dd357829003601f168201915b505050505090509450945094509450509193509193565b828054600181600116156101000203166002900490600052602060002090601f016020900481019282601f10610e4857805160ff1916838001178555610e76565b82800160010185558215610e76579182015b82811115610e75578251825591602001919060010190610e5a565b5b509050610e839190610f37565b5090565b60a06040519081016040528060008152602001606081526020016000815260200160008152602001600081525090565b828054600181600116156101000203166002900490600052602060002090601f016020900481019282601f10610ef857805160ff1916838001178555610f26565b82800160010185558215610f26579182015b82811115610f25578251825591602001919060010190610f0a565b5b509050610f339190610f37565b5090565b610f5991905b80821115610f55576000816000905550600101610f3d565b5090565b9056fea165627a7a72305820d8970bdf2f5c97a3c531322dcedcc71e60196e766222f698b6e1c5dbe90f71010029";

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    new web3.eth.Contract(_abiArray).deploy({data: _bin}).estimateGas(function(err, gas) {
        if(!err) console.log(">> gas: "+ gas);
    });
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 2000000}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })

    console.log("---> The contract deployed to: " + deployed.options.address)
}

deploy()

Overwriting src/ManagerDeploy.js


In [87]:
!node src/ManagerDeploy.js

Deploying the contract from 0x8e71085FfEFcCb88a4B54D4c93A866DB7928A486
>> gas: 1132561
hash: 0xb4577903a5789819d48863f3e2b2414369fb7085606f8e551fe956647cccd208
---> The contract deployed to: 0xAe2f94d9E2D48401B13D72381097Bcbc4a50a4c2


In [88]:
%%writefile src/ManagerUse.js
var Web3=require('web3');
var fs=require('fs');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8445"));
}
var _abiArray =[{"constant":false,"inputs":[{"name":"user","type":"address"}],"name":"updateStatus","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"user","type":"address"},{"name":"ordernum","type":"uint256"},{"name":"product","type":"string"},{"name":"count","type":"uint256"},{"name":"_cost","type":"uint256"}],"name":"Order","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":true,"inputs":[{"name":"user","type":"address"}],"name":"getsOrderWithAddr","outputs":[{"name":"","type":"uint256"},{"name":"","type":"string"},{"name":"","type":"uint256"},{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"ownerBalance","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[{"name":"odnum","type":"uint256"}],"name":"getOrderWithNum","outputs":[{"name":"","type":"uint256"},{"name":"","type":"string"},{"name":"","type":"uint256"},{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"}];

async function doIt() {
    var manage= new web3.eth.Contract(_abiArray, '0xAe2f94d9E2D48401B13D72381097Bcbc4a50a4c2');
    const accounts = await web3.eth.getAccounts();
    //console.log("Account: " + accounts[0]);
    
    //주문 1,2,3
    
    manage.methods.Order(accounts[2],306,"burger",12,10000).estimateGas(function(err, gas) {
        if(!err) console.log(">> gas: "+ gas);
    });
    await manage.methods.Order(accounts[1],305,"water",12,12000).send({from: accounts[0], gas: 187502});
    await manage.methods.Order(accounts[2],306,"burger",12,10000).send({from: accounts[0], gas: 188642});
    await manage.methods.Order(accounts[3],307,"paper",9,28000).send({from: accounts[0], gas: 188630});
    
    // 금액 출력
    manage.methods.ownerBalance().call().then(function(value) {console.log("sum of order",value)});
    
    //2번 계정의 주문 내역 출력
    manage.methods.getsOrderWithAddr(accounts[1]).call().then(function(value) {console.log("second account's...\n orderNumber:",value[0],"\n product:",value[1],"\n value:",value[2],"\n status:",value[3],"\n\n")});
    
    //주문 번호로 주문 내역 출력
    //console.log("------")
    manage.methods.getOrderWithNum(305).call().then(function(value) {console.log("account's...\n orderNumber:",value[0],"\n product:",value[1],"\n value:",value[2],"\n status:",value[3])});
    manage.methods.getOrderWithNum(306).call().then(function(value) {console.log("account's...\n orderNumber:",value[0],"\n product:",value[1],"\n value:",value[2],"\n status:",value[3])});
    manage.methods.getOrderWithNum(307).call().then(function(value) {console.log("account's...\n orderNumber:",value[0],"\n product:",value[1],"\n value:",value[2],"\n status:",value[3])});
    
    
    //컨트랙의 잔고 조회
    manage.methods.getBalance().call().then(function(value) {console.log("balance of contract:",value)});
    
    //주문 배송 후 delivered로 출력
    await manage.methods.updateStatus(accounts[1]).send({from: accounts[0], gas: 122209});
    
    //2번 결과 다시 출력
    manage.methods.getsOrderWithAddr(accounts[1]).call().then(function(value) {console.log("second account's...\n orderNumber:",value[0],"\n product:",value[1],"\n value:",value[2],"\n status:",value[3])});
    
    
}

doIt();


Overwriting src/ManagerUse.js


In [89]:
!node src/ManagerUse.js

>> gas: 187566
account's...
 orderNumber: 305 
 product: water 
 value: 12000 
 status: Ordered
sum of order 50000
second account's...
 orderNumber: 305 
 product: water 
 value: 12000 
 status: Ordered 


account's...
 orderNumber: 306 
 product: burger 
 value: 10000 
 status: Ordered
account's...
 orderNumber: 307 
 product: paper 
 value: 28000 
 status: Ordered
balance of contract: 0
second account's...
 orderNumber: 305 
 product: water 
 value: 12000 
 status: Delivered
